In [ ]:
async def finance_agent_generator(query: str):
    # Create the graph and initialize state (replace with your actual logic)
    graph = create_finance_agent_graph()
    initial_state = AgentState(
        query=query,
        processed_query="",
        use_graph_db=False,
        graph_db_result=[],
        expanded_queries=[],
        subgraphs_to_execute=3,
        subgraph_results={},
        final_response=""
    )

    current_state = initial_state

    # Simulate streaming updates
    async for chunk in graph.astream(current_state, stream_mode="updates"):
        for node, node_state in chunk.items():
            if isinstance(node_state, dict):
                final_response = node_state[node].get("final_response")
                
                # Debug: Print the final response to check its value
                print(f"Node: {node}, Final Response: {final_response}")

                if final_response:
                    yield f"data: {json.dumps({'type': 'final', 'content': final_response})}\n\n"
                else:
                    yield f"data: {json.dumps({'type': 'step', 'content': node})}\n"
            else:
                print(f"Unexpected state type for node {node}: {type(node_state[node])}.")

    # Ensuring the final response is properly sent
    if current_state.final_response:
        yield f"data: {json.dumps({'type': 'final', 'content': current_state.final_response})}\n\n"
    else:
        print("Final response is missing or empty")
        yield f"data: {json.dumps({'type': 'final', 'content': 'Final response placeholder'})}\n\n"
    
    yield "data: [DONE]\n\n"